Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import os

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
data_root = './raw_picture' # Change me to store data elsewhere

pickle_file = 'notMNIST.pickle'

with open(os.path.join(data_root,pickle_file), 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

print (train_labels.shape)

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

(200000,)
Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [15]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 100000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [16]:
num_steps = 7800

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 21.920757
Training accuracy: 4.5%
Validation accuracy: 7.0%
Loss at step 100: 2.477068
Training accuracy: 70.8%
Validation accuracy: 70.7%
Loss at step 200: 2.033210
Training accuracy: 73.8%
Validation accuracy: 73.2%
Loss at step 300: 1.802619
Training accuracy: 75.0%
Validation accuracy: 73.9%
Loss at step 400: 1.649534
Training accuracy: 75.6%
Validation accuracy: 74.6%
Loss at step 500: 1.537196
Training accuracy: 76.1%
Validation accuracy: 75.2%
Loss at step 600: 1.449408
Training accuracy: 76.4%
Validation accuracy: 75.5%
Loss at step 700: 1.377781
Training accuracy: 76.6%
Validation accuracy: 75.7%
Loss at step 800: 1.317629
Training accuracy: 76.9%
Validation accuracy: 76.1%
Loss at step 900: 1.266008
Training accuracy: 77.1%
Validation accuracy: 76.2%
Loss at step 1000: 1.221018
Training accuracy: 77.4%
Validation accuracy: 76.4%
Loss at step 1100: 1.181418
Training accuracy: 77.5%
Validation accuracy: 76.5%
Loss at step 1200: 1.146240
Training accu

Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [0]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [0]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0 : 16.8091
Minibatch accuracy: 12.5%
Validation accuracy: 14.0%
Minibatch loss at step 500 : 1.75256
Minibatch accuracy: 77.3%
Validation accuracy: 75.0%
Minibatch loss at step 1000 : 1.32283
Minibatch accuracy: 77.3%
Validation accuracy: 76.6%
Minibatch loss at step 1500 : 0.944533
Minibatch accuracy: 83.6%
Validation accuracy: 76.5%
Minibatch loss at step 2000 : 1.03795
Minibatch accuracy: 78.9%
Validation accuracy: 77.8%
Minibatch loss at step 2500 : 1.10219
Minibatch accuracy: 80.5%
Validation accuracy: 78.0%
Minibatch loss at step 3000 : 0.758874
Minibatch accuracy: 82.8%
Validation accuracy: 78.8%
Test accuracy: 86.1%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [9]:
# 构建一个隐层的神经网络
# 第一步：建构图结构
def add_layer(inputs, in_size, out_size, activation_function=None):
    weights = tf.Variable(tf.truncated_normal([in_size, out_size]))
    bias = tf.Variable(tf.zeros([1,out_size])+0.1)
    wx_plus_b = tf.matmul(inputs, weights) + bias
    if activation_function is None:
        outputs = wx_plus_b
    else:
        outputs = activation_function(wx_plus_b)
    return weights, bias, outputs


def tf_nn_accuracy(predictions, labels):
    correct_prediction = tf.equal(tf.argmax(predictions,1),tf.argmax(labels,1))       #高维度的
    acuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))    #要用reduce_mean
    return acuracy


graph = tf.Graph()
with graph.as_default():
    
    batch_size = tf.placeholder(tf.int32, [])
    
    input_x = tf.placeholder(tf.float32, [None, image_size*image_size])
    input_y = tf.placeholder(tf.float32, [None, num_labels])
    
    # hidden layer
    hl_weights, hl_bias, hl_outpus = add_layer(input_x, image_size*image_size, 1024, tf.nn.relu)
    # output layer
    ol_weights, ol_bias, ol_outputs = add_layer(hl_outpus, 1024, 10) # 隐层的输出作为下一层的输入
    
    # loss
    logits = tf.matmul(hl_outpus, ol_weights) + ol_bias
    
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=input_y, logits=ol_outputs))
    
     # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
  
    # 用于保存模型的函数
    saver = tf.train.Saver()
    
    y_pre = tf.nn.softmax(logits)
    
    _accuracy = tf_nn_accuracy(y_pre, input_y)

In [10]:
steps = 1000
_batch_size=128
model_save_path = './tf_nn_model_file'
train_subset = 100000
# 参考资料：https://blog.csdn.net/wlzzhcsdn/article/details/78203991?reload

print (train_labels.shape)
train_dataset = train_dataset[:train_subset, :]
train_labels = train_labels[:train_subset]

with tf.Session(graph=graph) as session:
    init = tf.initialize_all_variables()
    print('Initialized')
    # tf_nn_train_labels.shape[0]：样本数量
    for step in range(steps):
        
        session.run(init)
        offset = (step * _batch_size) % (train_labels.shape[0] - _batch_size)
        # Generate a minibatch.

        batch_dataset = train_dataset[offset:(offset + _batch_size), :]
        batch_labels = train_labels[offset:(offset + _batch_size), :]
        feed_dict = {input_x : batch_dataset, input_y : batch_labels, batch_size: _batch_size}
        
        _, l, train_accuracy = session.run([optimizer, loss, _accuracy], feed_dict=feed_dict)
        if (step % 100 == 0):
              print("Minibatch loss at step %d: %f, train accuracy: %f" % (step, l, train_accuracy))


valid_loss, valid_accuracy = session.run([loss, _accuracy], feed_dict=
                                              {input_x: valid_dataset, input_y: valid_labels, batch_size: valid_dataset.shape[0]})
print ("Validation loss: %.1f%%, Validation accuracy: %f" % (valid_loss, valid_accuracy))
               
print ("test accuracy {0}".format(session.run(_accuracy, feed_dict={
            input_x: test_dataset, input_y: test_labels, batch_size:test_dataset.shape[0]})))        

(100000, 10)
Initialized
Minibatch loss at step 0: 346.648132, train accuracy: 0.132812
Minibatch loss at step 100: 430.980316, train accuracy: 0.070312
Minibatch loss at step 200: 332.568726, train accuracy: 0.125000
Minibatch loss at step 300: 281.746002, train accuracy: 0.101562
Minibatch loss at step 400: 417.263000, train accuracy: 0.148438
Minibatch loss at step 500: 361.332092, train accuracy: 0.031250
Minibatch loss at step 600: 252.233505, train accuracy: 0.140625
Minibatch loss at step 700: 356.446716, train accuracy: 0.132812
Minibatch loss at step 800: 360.138733, train accuracy: 0.078125
Minibatch loss at step 900: 284.247437, train accuracy: 0.109375


RuntimeError: Attempted to use a closed Session.